# Part-I: Zip-code wise sea level risk

In [ ]:
!pip install bokeh
!pip install gmaps

In [ ]:
import pandas as pd
import numpy as np

# Fetching Dataset
slr_zip = pd.read_csv("Datasets/SLR_Zipcode.csv",index_col=0)

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.plotting import gmap
from bokeh.models import GMapOptions, ColorBar, HoverTool, ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.palettes import Plasma256 as palette
from bokeh.palettes import Turbo256 as palette1
from bokeh.plotting import figure, output_file, save
from bokeh.tile_providers import get_provider, WIKIMEDIA, ESRI_IMAGERY
from pyproj import Proj, transform

import warnings

warnings.filterwarnings("ignore")
pd.set_option("max_columns", 30)

# output_notebook()

In [ ]:
def plotSeaLevelRisk(df,city='',state=''):
    
    """
    Function to add Flood risk due to Sea Levl Rise data-layer to the Maps(GMap/WIKIMEDIA).
    
    Parameters:
    df(Pandas.dataframe object): (@Required)Dataset containing latitude, longitude and variable to be plotted.
    city: (@Optional)Name of the city to filter results.
    state: (@Optional)(@Required if city parameter is passed)Name of the state to filter results.
    
    Returns:
    p(plot object): Returns a plot with added data-layer.
        
    """
    
    if state in ['Alaska', 'Hawaii']:
            print(f"No data available for {state}.\nPlease refer the complete map.")
            return
    
    def plot(df, lat, lng, zoom=8, size=2,map_type='roadmap'):
        
        """
        Function to add data layer to Google Maps.

        Parameters:
        df(Pandas.dataframe object): Dataset containing latitude, longitude and variable to be plotted.
        lat(float),lng(float): Latitude and Longitude values required to define center point of the Map.
        zoom(int): Zoom level of the map. Default value is 8.
        size(int): Radius of the points plotted.
        map_type(String): Type of map to be displayed(roadmap/hybrid/terrain/satellite)

        """
        
        # Fetching Gmaps API key
        with open('APIKey.txt') as f:
            api_key = f.readline()
            f.close
        
        # Defining gmap object
        gmap_options = GMapOptions(lat=lat, lng=lng, map_type=map_type, zoom=zoom)
        
        # Defining information pop-up
        hover = HoverTool(
            tooltips = [
                ('Zip', '@zip'),
                ('Risk Score', '@avg_risk_score_all'),
            ]
        )
        
        # Defining gmap object using bokeh library
        p = gmap(api_key, gmap_options, title='Flood risk due to Sea Level Rise', 
                 width=950, height=700,
                 tools=[hover, 'reset', 'wheel_zoom', 'pan'])
        
        # Defining data source
        source = ColumnDataSource(df)
        
        # Defining and plotting color bar according to the risk-score
        maxRisk = max(df.avg_risk_score_all.values)
        minRisk = min(df.avg_risk_score_all.values)

        mapper = linear_cmap('avg_risk_score_all', palette, minRisk, maxRisk)

        center = p.circle('lng', 'lat', size=size, color=mapper, source=source)

        color_bar = ColorBar(color_mapper=mapper['transform'], 
                             location=(0,0))

        p.add_layout(color_bar, 'right')
        p.title.text_font_size = "12pt"
        show(p)
        return p
    
    # State lists to define zoom level of the map.
    largeStates = ["Texas","California","Montana","New Mexico","Arizona","Nevada","Colorado","Wyoming","Oregon","Idaho",
               "Utah","Kansas","Minnesota","Nebraska","South Dakota","North Dakota","Missouri","Oklahoma",
               "Washington","Georgia","Michigan"]

    midStates = ["Iowa","Illinois","Wisconsin","Florida","Arkansas","Alabama","North Carolina",
             "New York","Mississippi","Pennsylvania","Louisiana","Tennessee","Ohio","Kentucky","Virginia",
             "Indiana","Maine","South Carolina","West Virginia","Maryland","Vermont","New Hampshire"]

    smallStates = ["Massachusetts","New Jersey","Hawaii","Connecticut","Delaware","Rhode Island"]
    
    # Plotting map according to the passed state.
    if city == '' and state !='':
        
        state = state.title()
        df = df[df.state_name==state]
        states = pd.read_csv("Datasets/States.csv")
        lat = states[states.name==state][['latitude','longitude']].values[0][0]
        lng = states[states.name==state][['latitude','longitude']].values[0][1]
        
        if state in largeStates:
            p = plot(df, lat, lng, zoom=6,size=4)
        elif state in midStates:
            p = plot(df, lat, lng, zoom=7,size=4)
        else:
            p = plot(df, lat, lng, zoom=8,size=4)
    
    # Plotting map according to the passed city and state.
    elif city != '' and state !='':
        
        city = city.capitalize()
        state = state.capitalize()
        df = df[(df.city==city)&(df.state_name==state)]
        lat = df[['lat','lng']].values[0][0]
        lng = df[['lat','lng']].values[0][1]
        p = plot(df, lat, lng,zoom=11,size=8)
    
    # Plotting nationwide map for all the zip-codes in the US.
    elif city == '' and state == '':
        
        wikimedia = get_provider(WIKIMEDIA)
        
        inProj = Proj(init='epsg:3857')
        outProj = Proj(init='epsg:4326')

        maxRisk = max(df.avg_risk_score_all.values)
        minRisk = min(df.avg_risk_score_all.values)

        us_lon1, us_lat1 = transform(outProj,inProj,-140,10)
        us_lon2, us_lat2 = transform(outProj,inProj,-50,55)
        
        # Defining plot using wikimedia map provider
        p = figure(plot_width=1420, plot_height=650,
                   x_range=(us_lon1, us_lon2), y_range=(us_lat1, us_lat2),
                   x_axis_type="mercator", y_axis_type="mercator",
                  tooltips=[
                            ("Zipcode", "@zip"), ("Risk Score","@avg_risk_score_all")])

        p.add_tile(wikimedia)
        
        # Defining and plotting color bar according to the risk-score
        mapper = linear_cmap('avg_risk_score_all', palette, minRisk, maxRisk)

        p.circle(x="MercatorX", y="MercatorY",
                 size=2,
                 color=mapper,
                 fill_alpha=0.3,
                 source=df)

        color_bar = ColorBar(color_mapper=mapper['transform'], 
                                 location=(0,0))

        p.add_layout(color_bar, 'right')

        show(p)
    
    elif city != '' and state == '':
        print("Please pass state name along with the city name to view the results.")
        return
        
    return p

In [ ]:
p = plotSeaLevelRisk(slr_zip, city="Boston", state="Massachusetts")

# Saving figure
# output_file(filename="FloodRisk-SLR.html");
# save(p);

# Part II: City wise sea-level rise

In [ ]:
from ipywidgets.embed import embed_minimal_html
import gmaps

# Fetching dataset
noaa = pd.read_csv("Datasets/NOAA.csv")

# Defining dictionary for NOAA scenarios
scenarios = {1:'0.3 - MED', 2:'0.3 - LOW', 3:'0.3 - HIGH', 4:'0.5 - MED', 5:'0.5 - LOW',
       6:'0.5 - HIGH', 7:'1.0 - MED', 8:'1.0 - LOW', 9:'1.0 - HIGH', 10:'1.5 - MED',
       11:'1.5 - LOW', 12:'1.5 - HIGH', 13:'2.0 - MED', 14:'2.0 - LOW', 15:'2.0 - HIGH',
       16:'2.5 - MED', 17:'2.5 - LOW', 18:'2.5 - HIGH'}

In [ ]:
def plotSeaLevel(df, lat = 39.8097343, lng = -98.5556199, scenario='1.5 - MED', zoom=4, map_type='HYBRID'):
    
    """
    Function to add Sea Level Rise data-layer to the Google Maps.
    
    Parameters:
    df(Pandas.dataframe object): (@Required)Dataset containing latitude, longitude and variable to be plotted.
    lat(float),lng(float): (@Optional)Latitude and Longitude values required to define center point of the Map. 
                                      Default lat = 39.8097343 and lng = -98.5556199 gives the center of the US.
    zoom(int): (@Optional)Zoom level of the map. Default value is 4.
    scenario(String): (@Optional)NOAA scenario to be plotted. Default value is '1.5 - MED'.
    map_type(String): (@Optional)Type of map to be displayed(ROADMAP/HYBRID/TERRAIN/SATELLITE). 
                                 Default value is 'HYBRID'.
    
    Returns:
    fig(gmap plot object): Returns a gmap plot with added data-layer.
        
    """
    
    # Fetching Gmap API key
    with open('APIKey.txt') as f:
        api_key = f.readline()
        f.close

    gmaps.configure(api_key=api_key)
    
    # Data processing
    df = df[['Site', 'Latitude', 'Longitude', 'Scenario', 'RSL in 2020 (cm)']]
    df = df.rename(columns={"RSL in 2020 (cm)": "RSL"})
    df = df[df.Scenario==scenario]
    df.reset_index(drop=True,inplace=True)
    
    # Defining information pop-up
    site_info = []
    for i in range(len(df[['Site','RSL']].values)):
        city = df[['Site','RSL']].values[i][0]
        RSL = df[['Site','RSL']].values[i][1]
        site_info.append({'City':city,'RSL':RSL})
    
    template = """
    <dl>
    <dt>City</dt><dd>{City}</dd>
    <dt>Sea Level Rise(cm)</dt><dd>{RSL}</dd>
    </dl>
    """
    info = [template.format(**site) for site in site_info]
    
    # Adding data-layer and plotting map.
    SLR_layer = gmaps.symbol_layer(df[['Latitude', 'Longitude']], fill_color='yellow', 
                                   stroke_color='yellow', stroke_opacity=0.7, 
                                   fill_opacity=0.8,scale=3,info_box_content=info)
    fig = gmaps.figure(layout={'width':'900px','height':'700px'},
                       map_type=map_type,center=(lat,lng),zoom_level=zoom)
    fig.add_layer(SLR_layer)
    return fig

In [ ]:
fig = plotSeaLevel(noaa,scenario=scenarios[7])
fig

# Saving figure
# embed_minimal_html('SLR.html', views=[fig])